In [108]:
import csv
import operator
from itertools import combinations

                
path = 'E:\\2023\\데이터마이닝\\store_data(2).csv' ## 파일 저장 위치

pre_lst = [] # k-basket list
pre_dict = {} # (k-1) -basket dictionary
sub_dict = {} # k basket dictionary
first_dict = {} # interest 계산 시 사용

support_threshold = 300
total = 7501 ## 전체 데이터 수
min_support = support_threshold/total
min_confidence = 0.2 # 최저 confidence값

def tuple_to_list(tuple_list):
    lst = []
    for x in tuple_list:
        for y in x:
            if y not in lst:
                lst.append(y)
    return lst

## 새로운 집합 생성
def candidate(subset, frequent_list):
    if subset > 2:
        new_frequent_list = tuple_to_list(frequent_list)
    else:
        new_frequent_list = frequent_list
    
    new_set = list(combinations(new_frequent_list, subset))
    
    return new_set

## support 미만 제거
def prune(basket_prn, support):   
    for key in list(basket_prn.keys()):
        if basket_prn[key] < support:
            del basket_prn[key] 

    return basket_prn

## 파일 읽어오는 함수
def FileRead(subset, pre_items):
    new_basket_dict = {}
    
    with open(path, newline = "") as file:
        reader = csv.reader(file)

        for idx, row in enumerate(reader):
            if subset == 1: ## 최초, 집합 1개일 경우
                for element in row:
                    if element not in new_basket_dict:
                        new_basket_dict[element] = 1
                        first_dict[element] = 1
                    else:
                        new_basket_dict[element] += 1
                        first_dict[element] += 1
                
            
            else:
                ## 집합 생성                
                basket_lst = candidate(subset, pre_items)

                for items in basket_lst:
                    if ((set(items)&set(row))==set(items)):
                        if items not in new_basket_dict:
                            new_basket_dict[items] = 1
                        else:
                            new_basket_dict[items] += 1
    return new_basket_dict

def cal_confidence(subset, pre_dict, key, NOC, support):
    if subset > 2:
        comb = list(combinations(key, subset-1))
    elif subset == 2:
        comb = list(pre_dict.keys())
    
    for X in comb:
        if subset > 2:
            for P in pre_dict:
                if ((set(X)&set(P))==set(X)): 
                    Y = list(set(key) - set(X))
                    perc = NOC/pre_dict[P]
                    
                    sup.append(support)
                    ant.append(X)
                    conse.append(Y)
                    confi.append(perc)
                    P_Y.append(first_dict[Y[0]]/total)
                    
                    if perc > min_confidence: ## min_confidence 이상일 경우만 출력
                        print(f'confidence {X} -> {Y} : {perc}')
                    
        elif subset == 2:
            for P in key:
                if X==P:
                    Y = list(key).copy()
                    Y.remove(X)
                    perc = NOC/pre_dict[P]
                    
                    sup.append(support)
                    ant.append(X)
                    conse.append(Y)
                    confi.append(perc)
                    print(Y)
                    P_Y.append(first_dict[Y[0]]/total)
                    
                    if perc > min_confidence:
                        print(f'confidence {X} -> {Y} : {NOC/pre_dict[P]}')

def cal_support(subset, bsk_dict, pre_dict):
    for key in bsk_dict.keys():
        ## support 출력
        support = bsk_dict[key]/total

        print(f'\nsupport {key}: {support}')
        cal_confidence(subset, pre_dict, key, bsk_dict[key], support) 
        
## supoort threshold 값을 만족할때까지 반복 (subset, support, k-basket list, (k-1)-basket dict, (k)-basket dict)
def apriori(subset, support, pre_lst, pre_dict, freq_dict): 
    basket_dict = FileRead(subset, pre_lst)

    basket_dict = prune(basket_dict, support)
    frequent_itemset = list(basket_dict.keys()) ## frequent한 itemset
    
    if (basket_dict.keys()):## basket의 key가 남아있는 동안 반복
        apriori(subset+1, support, frequent_itemset, freq_dict, basket_dict) # (subset, support, frequent_itemset, k-basket, (k+1) basket)
    
    else:
        cal_support(subset-1, freq_dict, pre_dict) #(subset, k-1, k)
    

In [81]:
## 데이터 프레임 저장
import pandas as pd
count = 0
data = pd.DataFrame({'antecedents' : {}, 'consequents' : {}, 'support':{}, 'confidence' :{}})

ant = []
conse = []
sup = []
confi = []
P_Y = []

apriori(1, support_threshold, pre_lst, pre_dict, sub_dict)

data['support'] = sup
data['antecedents'] = ant
data['consequents'] = conse
data['confidence'] = confi
data['P_Y'] = P_Y

data.sort_values(by = ['support'])
data['interest'] = abs(data['confidence'] - data['P_Y'])

data.to_csv('support_threshold100.csv', index = False)


support ('mineral water', 'eggs', 'spaghetti'): 0.014264764698040262
confidence ('mineral water', 'eggs') -> ['spaghetti'] : 0.2801047120418848
confidence ('mineral water', 'spaghetti') -> ['eggs'] : 0.23883928571428573
confidence ('eggs', 'spaghetti') -> ['mineral water'] : 0.3905109489051095

support ('mineral water', 'milk', 'spaghetti'): 0.01573123583522197
confidence ('mineral water', 'milk') -> ['spaghetti'] : 0.3277777777777778
confidence ('mineral water', 'spaghetti') -> ['milk'] : 0.26339285714285715
confidence ('milk', 'spaghetti') -> ['mineral water'] : 0.44360902255639095

support ('mineral water', 'spaghetti', 'ground beef'): 0.017064391414478068
confidence ('mineral water', 'spaghetti') -> ['ground beef'] : 0.2857142857142857
confidence ('mineral water', 'ground beef') -> ['spaghetti'] : 0.4169381107491857
confidence ('spaghetti', 'ground beef') -> ['mineral water'] : 0.43537414965986393

support ('mineral water', 'eggs', 'chocolate'): 0.013464871350486601
confidence ('m

In [82]:
data.head(10)

,antecedents,consequents,support,confidence,P_Y,interest
0,"(mineral water, eggs)",[spaghetti],0.014265,0.280105,0.174110,0.105995
1,"(mineral water, spaghetti)",[eggs],0.014265,0.238839,0.179709,0.059130
2,"(eggs, spaghetti)",[mineral water],0.014265,0.390511,0.238368,0.152143
3,"(mineral water, milk)",[spaghetti],0.015731,0.327778,0.174110,0.153668
4,"(mineral water, spaghetti)",[milk],0.015731,0.263393,0.129583,0.133810
5,"(milk, spaghetti)",[mineral water],0.015731,0.443609,0.238368,0.205241
6,"(mineral water, spaghetti)",[ground beef],0.017064,0.285714,0.098254,0.187461
7,"(mineral water, ground beef)",[spaghetti],0.017064,0.416938,0.174110,0.242828
8,"(spaghetti, ground beef)",[mineral water],0.017064,0.435374,0.238368,0.197006
9,"(mineral water, eggs)",[chocolate],0.013465,0.264398,0.163978,0.100420


In [86]:
data.sort_values(by = ['support'], ascending = False).head(10)

,antecedents,consequents,support,confidence,P_Y,interest
6,"(mineral water, spaghetti)",[ground beef],0.017064,0.285714,0.098254,0.187461
7,"(mineral water, ground beef)",[spaghetti],0.017064,0.416938,0.174110,0.242828
8,"(spaghetti, ground beef)",[mineral water],0.017064,0.435374,0.238368,0.197006
17,"(spaghetti, chocolate)",[mineral water],0.015865,0.404762,0.238368,0.166394
15,"(mineral water, spaghetti)",[chocolate],0.015865,0.265625,0.163978,0.101647
16,"(mineral water, chocolate)",[spaghetti],0.015865,0.301266,0.174110,0.127156
3,"(mineral water, milk)",[spaghetti],0.015731,0.327778,0.174110,0.153668
4,"(mineral water, spaghetti)",[milk],0.015731,0.263393,0.129583,0.133810
5,"(milk, spaghetti)",[mineral water],0.015731,0.443609,0.238368,0.205241
0,"(mineral water, eggs)",[spaghetti],0.014265,0.280105,0.174110,0.105995


In [93]:
## 데이터 프레임 저장
import pandas as pd
count = 0
data2 = pd.DataFrame({'antecedents' : {}, 'consequents' : {}, 'support':{}, 'confidence' :{}})

ant = []
conse = []
sup = []
confi = []
P_Y = []

apriori(1, support_threshold, pre_lst, pre_dict, sub_dict)

data2['support'] = sup
data2['antecedents'] = ant
data2['consequents'] = conse
data2['confidence'] = confi
data2['P_Y'] = P_Y

data2.sort_values(by = ['support'])
data2['interest'] = abs(data2['confidence'] - data2['P_Y'])

data2.to_csv('support_threshold200.csv', index = False)


support ('green tea', 'mineral water'): 0.03106252499666711
['mineral water']
confidence green tea -> ['mineral water'] : 0.23511604439959638
['green tea']

support ('mineral water', 'olive oil'): 0.027596320490601255
['olive oil']
['mineral water']
confidence olive oil -> ['mineral water'] : 0.4190283400809717

support ('burgers', 'eggs'): 0.02879616051193174
['eggs']
confidence burgers -> ['eggs'] : 0.3302752293577982
['burgers']

support ('mineral water', 'milk'): 0.04799360085321957
['milk']
confidence mineral water -> ['milk'] : 0.20134228187919462
['mineral water']
confidence milk -> ['mineral water'] : 0.37037037037037035

support ('frozen vegetables', 'spaghetti'): 0.027862951606452472
['spaghetti']
confidence frozen vegetables -> ['spaghetti'] : 0.2923076923076923
['frozen vegetables']

support ('mineral water', 'eggs'): 0.05092654312758299
['eggs']
confidence mineral water -> ['eggs'] : 0.21364653243847875
['mineral water']
confidence eggs -> ['mineral water'] : 0.2833827893

PermissionError: [Errno 13] Permission denied: 'support_threshold200.csv'

In [106]:
data2.sort_values(by = ['support'], ascending = False)

,antecedents,consequents,support,confidence,P_Y,interest
13,spaghetti,[mineral water],0.059725,0.343032,0.238368,0.104664
12,mineral water,[spaghetti],0.059725,0.250559,0.174110,0.076449
28,mineral water,[chocolate],0.052660,0.220917,0.163978,0.056939
29,chocolate,[mineral water],0.052660,0.321138,0.238368,0.082770
10,mineral water,[eggs],0.050927,0.213647,0.179709,0.033937
11,eggs,[mineral water],0.050927,0.283383,0.238368,0.045015
6,mineral water,[milk],0.047994,0.201342,0.129583,0.071760
7,milk,[mineral water],0.047994,0.370370,0.238368,0.132002
23,ground beef,[mineral water],0.040928,0.416554,0.238368,0.178185
22,mineral water,[ground beef],0.040928,0.171700,0.098254,0.073447


In [109]:
## 데이터 프레임 저장
import pandas as pd
count = 0
data3 = pd.DataFrame({'antecedents' : {}, 'consequents' : {}, 'support':{}, 'confidence' :{}})

ant = []
conse = []
sup = []
confi = []
P_Y = []

apriori(1, support_threshold, pre_lst, pre_dict, sub_dict)

data3['support'] = sup
data3['antecedents'] = ant
data3['consequents'] = conse
data3['confidence'] = confi
data3['P_Y'] = P_Y

data3.sort_values(by = ['support'])
data3['interest'] = abs(data3['confidence'] - data3['P_Y'])

data3.to_csv('support_threshold300.csv', index = False)


support ('mineral water', 'milk'): 0.04799360085321957
['milk']
confidence mineral water -> ['milk'] : 0.20134228187919462
['mineral water']
confidence milk -> ['mineral water'] : 0.37037037037037035

support ('mineral water', 'eggs'): 0.05092654312758299
['eggs']
confidence mineral water -> ['eggs'] : 0.21364653243847875
['mineral water']
confidence eggs -> ['mineral water'] : 0.28338278931750743

support ('mineral water', 'spaghetti'): 0.05972536995067324
['spaghetti']
confidence mineral water -> ['spaghetti'] : 0.2505592841163311
['mineral water']
confidence spaghetti -> ['mineral water'] : 0.3430321592649311

support ('mineral water', 'ground beef'): 0.040927876283162246
['ground beef']
['mineral water']
confidence ground beef -> ['mineral water'] : 0.41655359565807326

support ('mineral water', 'chocolate'): 0.05265964538061592
['chocolate']
confidence mineral water -> ['chocolate'] : 0.220917225950783
['mineral water']
confidence chocolate -> ['mineral water'] : 0.32113821138211

In [110]:
data3.sort_values(by = ['support'], ascending = False)

,antecedents,consequents,support,confidence,P_Y,interest
4,mineral water,[spaghetti],0.059725,0.250559,0.174110,0.076449
5,spaghetti,[mineral water],0.059725,0.343032,0.238368,0.104664
8,mineral water,[chocolate],0.052660,0.220917,0.163978,0.056939
9,chocolate,[mineral water],0.052660,0.321138,0.238368,0.082770
2,mineral water,[eggs],0.050927,0.213647,0.179709,0.033937
3,eggs,[mineral water],0.050927,0.283383,0.238368,0.045015
0,mineral water,[milk],0.047994,0.201342,0.129583,0.071760
1,milk,[mineral water],0.047994,0.370370,0.238368,0.132002
6,mineral water,[ground beef],0.040928,0.171700,0.098254,0.073447
7,ground beef,[mineral water],0.040928,0.416554,0.238368,0.178185


In [125]:
n = []
for i in range(data3['support'].count()):
               n.append(300)
        
len(n)
data3['threshold'] = n


In [128]:
data3

,antecedents,consequents,support,confidence,P_Y,interest,threshold
0,mineral water,[milk],0.047994,0.201342,0.129583,0.071760,300
1,milk,[mineral water],0.047994,0.370370,0.238368,0.132002,300
2,mineral water,[eggs],0.050927,0.213647,0.179709,0.033937,300
3,eggs,[mineral water],0.050927,0.283383,0.238368,0.045015,300
4,mineral water,[spaghetti],0.059725,0.250559,0.174110,0.076449,300
5,spaghetti,[mineral water],0.059725,0.343032,0.238368,0.104664,300
6,mineral water,[ground beef],0.040928,0.171700,0.098254,0.073447,300
7,ground beef,[mineral water],0.040928,0.416554,0.238368,0.178185,300
8,mineral water,[chocolate],0.052660,0.220917,0.163978,0.056939,300
9,chocolate,[mineral water],0.052660,0.321138,0.238368,0.082770,300


In [127]:
data3.drop( ['num'], axis = 1, inplace = True)

In [121]:
data2['support'].count()

n = []
for i in range(data2['support'].count()):
               n.append(200)
        
len(n)
data2['threshold'] = n

data2

,antecedents,consequents,support,confidence,P_Y,interest,threshold
0,green tea,[mineral water],0.031063,0.235116,0.238368,0.003252,200
1,mineral water,[green tea],0.031063,0.130313,0.132116,0.001803,200
2,mineral water,[olive oil],0.027596,0.115772,0.065858,0.049914,200
3,olive oil,[mineral water],0.027596,0.419028,0.238368,0.180660,200
4,burgers,[eggs],0.028796,0.330275,0.179709,0.150566,200
5,eggs,[burgers],0.028796,0.160237,0.087188,0.073049,200
6,mineral water,[milk],0.047994,0.201342,0.129583,0.071760,200
7,milk,[mineral water],0.047994,0.370370,0.238368,0.132002,200
8,frozen vegetables,[spaghetti],0.027863,0.292308,0.174110,0.118198,200
9,spaghetti,[frozen vegetables],0.027863,0.160031,0.095321,0.064710,200


In [122]:
data['support'].count()

n = []
for i in range(data['support'].count()):
               n.append(100)
        
len(n)
data['threshold'] = n
data

,antecedents,consequents,support,confidence,P_Y,interest,threshold
0,"(mineral water, eggs)",[spaghetti],0.014265,0.280105,0.174110,0.105995,100
1,"(mineral water, spaghetti)",[eggs],0.014265,0.238839,0.179709,0.059130,100
2,"(eggs, spaghetti)",[mineral water],0.014265,0.390511,0.238368,0.152143,100
3,"(mineral water, milk)",[spaghetti],0.015731,0.327778,0.174110,0.153668,100
4,"(mineral water, spaghetti)",[milk],0.015731,0.263393,0.129583,0.133810,100
5,"(milk, spaghetti)",[mineral water],0.015731,0.443609,0.238368,0.205241,100
6,"(mineral water, spaghetti)",[ground beef],0.017064,0.285714,0.098254,0.187461,100
7,"(mineral water, ground beef)",[spaghetti],0.017064,0.416938,0.174110,0.242828,100
8,"(spaghetti, ground beef)",[mineral water],0.017064,0.435374,0.238368,0.197006,100
9,"(mineral water, eggs)",[chocolate],0.013465,0.264398,0.163978,0.100420,100


In [130]:
result = pd.concat([data, data2])

In [131]:
result

,antecedents,consequents,support,confidence,P_Y,interest,threshold
0,"(mineral water, eggs)",[spaghetti],0.014265,0.280105,0.174110,0.105995,100
1,"(mineral water, spaghetti)",[eggs],0.014265,0.238839,0.179709,0.059130,100
2,"(eggs, spaghetti)",[mineral water],0.014265,0.390511,0.238368,0.152143,100
3,"(mineral water, milk)",[spaghetti],0.015731,0.327778,0.174110,0.153668,100
4,"(mineral water, spaghetti)",[milk],0.015731,0.263393,0.129583,0.133810,100
...,...,...,...,...,...,...,...
43,chocolate,[milk],0.032129,0.195935,0.129583,0.066352,200
44,french fries,[spaghetti],0.027596,0.161466,0.174110,0.012644,200
45,spaghetti,[french fries],0.027596,0.158499,0.170911,0.012411,200
46,eggs,[milk],0.030796,0.171365,0.129583,0.041782,200


In [134]:
result = pd.concat([result, data3])

In [135]:
result

,antecedents,consequents,support,confidence,P_Y,interest,threshold
0,"(mineral water, eggs)",[spaghetti],0.014265,0.280105,0.174110,0.105995,100
1,"(mineral water, spaghetti)",[eggs],0.014265,0.238839,0.179709,0.059130,100
2,"(eggs, spaghetti)",[mineral water],0.014265,0.390511,0.238368,0.152143,100
3,"(mineral water, milk)",[spaghetti],0.015731,0.327778,0.174110,0.153668,100
4,"(mineral water, spaghetti)",[milk],0.015731,0.263393,0.129583,0.133810,100
...,...,...,...,...,...,...,...
5,spaghetti,[mineral water],0.059725,0.343032,0.238368,0.104664,300
6,mineral water,[ground beef],0.040928,0.171700,0.098254,0.073447,300
7,ground beef,[mineral water],0.040928,0.416554,0.238368,0.178185,300
8,mineral water,[chocolate],0.052660,0.220917,0.163978,0.056939,300


In [139]:
result.sort_values(by = 'interest', ascending = False).head(10)

,antecedents,consequents,support,confidence,P_Y,interest,threshold
7,"(mineral water, ground beef)",[spaghetti],0.017064,0.416938,0.174110,0.242828,100
25,ground beef,[spaghetti],0.039195,0.398915,0.174110,0.224804,200
5,"(milk, spaghetti)",[mineral water],0.015731,0.443609,0.238368,0.205241,100
14,"(milk, chocolate)",[mineral water],0.013998,0.435685,0.238368,0.197316,100
8,"(spaghetti, ground beef)",[mineral water],0.017064,0.435374,0.238368,0.197006,100
6,"(mineral water, spaghetti)",[ground beef],0.017064,0.285714,0.098254,0.187461,100
3,olive oil,[mineral water],0.027596,0.419028,0.238368,0.180660,200
23,ground beef,[mineral water],0.040928,0.416554,0.238368,0.178185,200
7,ground beef,[mineral water],0.040928,0.416554,0.238368,0.178185,300
11,"(eggs, chocolate)",[mineral water],0.013465,0.405622,0.238368,0.167254,100
